In [1]:
from typing import Optional, Literal
from repsheet_backend.common import MEMBER_VOTES_TABLE, BILLS_TABLE, VOTES_HELD_TABLE, db_connect
from repsheet_backend.bills import BillId, BillIssues, BillSummary
from pydantic import BaseModel

In [ ]:
with open("prompts/summarize-member/001.txt", "r") as f:
    prompt_template = f.read()

class BillVotingRecord(BaseModel):
    summary: str
    billID: str
    billNumber: str
    voted: Literal["yea", "nay", "abstain"]
    issues: BillIssues


MEMBER_BILL_VOTING_QUERY = f"""
SELECT 
    b.[Bill ID] AS bill_id,
    b.[Bill Number] AS bill_number,
    b.[Summary] AS full_summary,
    mv.[Member Voted] AS voted
FROM {MEMBER_VOTES_TABLE} AS mv
JOIN {VOTES_HELD_TABLE} v
    ON mv.[Vote ID] = v.[Vote ID]
JOIN {BILLS_TABLE} AS b
    ON v.[Bill ID] = b.[Bill ID]
WHERE
    mv.[Member ID] = :member_id
"""

def get_member_voting_record(member_id: str) -> list[BillVotingRecord]:
    with db_connect() as db:
        rows = db.execute(MEMBER_BILL_VOTING_QUERY, {"member_id": member_id}).fetchall()

    voting_record: list[BillVotingRecord] = []
    for row in rows:
        full_summary = BillSummary.model_validate_json(row["full_summary"])
        voted = row["voted"].lower() if row["voted"] else "abstain" 
        voting_record.append(
            BillVotingRecord(
                summary=full_summary.summary,
                billID=row["bill_id"],
                billNumber=row["bill_number"],
                voted=voted,
                issues=full_summary.issues,
            )
        )
    return voting_record



def get_member_summarisation_prompt(member_id: str) -> str:
     = get_member_voting_record(bill)
    if xml_path is None:
        return None
    with open(xml_path, "r") as f:
        xml_text = f.read()
    return prompt_template.replace("{{RAW_INPUT_DATA}}", xml_text)